O presente exercício é uma reprodução de uma tarefa do curso de ciência de dados da univesp

# COM530 - Visão Computacional

## Semana 5 - Segmentação
## Videoaula 15 - Prática de Segmentação Semântica com CNN

- Importando os pacotes e imagens
- Analisando as imagens
- Preparando os dados
- Construindo uma rede convolucional Unet
- Treinando a rede
- Avaliando os resultados

Observações: 
- Ativar a GPU (Menu [Runtime], opção [Change runtime type])
- Sem GPU, a execução deste caderno pode levar mais de 40 minutos.


#Importar os pacotes e carregar os dados.

O conjunto de dados utilizado nesta aula foi construído a partir de imagens de Microscopia Eletrônica, as quais estão disponíveis em: https://www.epfl.ch/labs/cvlab/data/data-em/

Essas imagens estão em alta-resolução e foram segmentadas em mitocôndrias e sinapses.

A partir dessas imagens, foram geradas "sub-imagens" 256x256 a partir das regiões das imagens originais. Essas imagens 256x256 são as utilizadas nesta aula.

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.utils import normalize
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
import glob

#Carregando as imagens

O conjunto de dados utilizado nesta aula foi construído a partir de imagens de Microscopia Eletrônica, as quais estão disponíveis em: https://www.epfl.ch/labs/cvlab/data/data-em/

Essas imagens estão em alta-resolução e foram segmentadas em mitocôndrias e sinapses.

A partir dessas imagens, foram geradas "sub-imagens" 256x256 a partir das regiões das imagens originais. Essas imagens 256x256 são as utilizadas nesta aula.

In [ ]:
#!wget https://www.epfl.ch/labs/cvlab/data/data-em/

!wget https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/volumedata.tif

Para ler arquivos .tif no Pandas (uma biblioteca popular para análise de dados em Python), você pode seguir algumas abordagens diferentes, dependendo das suas necessidades específicas. Vou apresentar três maneiras comuns de lidar com arquivos .tif em Python:

Usando a biblioteca GDAL (Geospatial Data Abstraction Library):
O GDAL é uma biblioteca poderosa para trabalhar com dados geoespaciais, incluindo arquivos .tif.
Primeiro, você precisa instalar o GDAL no Python. Você pode fazer isso usando o Anaconda com o seguinte comando:
conda install gdal
Ou, se preferir, pode usar o pip:
pip install gdal

Em seguida, carregue o pacote gdal do módulo osgeo no seu console Python ou script:
Python

from osgeo import gdal
Código gerado por IA. Examine e use com cuidado. Mais informações em perguntas frequentes.
Para abrir um arquivo .tif no modo somente leitura e salvar os dados em uma variável, você pode fazer o seguinte:
Python

caminho_do_arquivo = 'caminho/do/arquivo.tif'
raster = gdal.Open(caminho_do_arquivo, gdal.GA_ReadOnly)
Código gerado por IA. Examine e use com cuidado. Mais informações em perguntas frequentes.
Você também pode copiar a transformação geográfica para uma variável, que pode ser útil posteriormente:
Python

transformacao_geografica = raster.GetGeoTransform()
Código gerado por IA. Examine e use com cuidado. Mais informações em perguntas frequentes.
Usando a biblioteca georasters:
A biblioteca georasters permite converter arquivos .tif em GeoPandas DataFrames ou Pandas DataFrames.
Primeiro, instale a biblioteca com o seguinte comando:
pip install georasters

Em seguida, importe o módulo georasters e carregue o arquivo .tif:
Python

import georasters as gr
import pandas as pd

meu_raster = 'caminho/do/arquivo.tif'
elevacao = gr.from_file(meu_raster)
# Agora você pode usar 'elevacao' como um DataFrame
Código gerado por IA. Examine e use com cuidado. Mais informações em perguntas frequentes.
Usando a biblioteca tifffile:
A biblioteca tifffile é útil para ler e escrever arquivos .tif em formato de matriz numpy.
Primeiro, instale a biblioteca com o seguinte comando:
pip install tifffile

Para ler um arquivo .tif em um array numpy, faça o seguinte:
Python

from tifffile import imread

caminho_do_arquivo = 'caminho/do/arquivo.tif'
imagem = imread(caminho_do_arquivo)
# Agora 'imagem' é um array numpy
Código gerado por IA. Examine e use com cuidado. Mais informações em perguntas frequentes.
Se você quiser salvar um array numpy como um arquivo .tif, use o seguinte:
Python

from tifffile import imwrite

meu_array_numpy = ...  # Seu array numpy aqui
imwrite('meu_arquivo.tif', meu_array_numpy, photometric='rgb')
Código gerado por IA. Examine e use com cuidado. Mais informações em perguntas frequentes.
Escolha a abordagem que melhor se adapta às suas necessidades e ao seu projeto! 🌟

pip install pylibtiff

PyLibTiff é um pacote que fornece:

um wrapper para a biblioteca libtiff para Python usando ctypes.

um módulo Python puro para ler e escrever arquivos TIFF e LSM. O As imagens são lidas como objetos para que seja possível abrir imagens que, de outra forma, não caberiam na memória RAM dos computadores. Ambos TIFF Faixas e blocos são suportados para armazenamento de dados de baixo nível.numpy.memmap

Existem muitos pacotes Python como PIL, FreeImagePy que suportam leitura e gravação de arquivos TIFF. O projeto PyLibTiff foi iniciado para ter uma maneira eficiente e direta de ler e gravar arquivos TIFF usando a biblioteca libtiff sem a necessidade de instalar qualquer desnecessário pacotes ou bibliotecas. O módulo Python puro foi criado para leitura arquivos TIFF "quebrados", como arquivos LSM que em alguns lugares usam interpretação diferente das etiquetas TIFF do que a especificada no TIFF documento de especificação. A biblioteca libtiff simplesmente falharia na leitura tais arquivos. Além disso, o módulo Python puro é mais memória eficiente à medida que os arrays são retornados como mapas de memória. Suporte para arquivos compactados ainda não está implementado.

tifffile.py por Christoph Gohlke é um excelente módulo para leitura de TIFF, bem como LSM, é tão rápido quanto libtiff.py usando numpy.

Exemplo de uso (wrapper libtiff)
>>> from libtiff import TIFF
>>> # to open a tiff file for reading:
>>> tif = TIFF.open('filename.tif', mode='r')
>>> # to read an image in the currect TIFF directory and return it as numpy array:
>>> image = tif.read_image()
>>> # to read all images in a TIFF file:
>>> for image in tif.iter_images(): # do stuff with image
>>> # to open a tiff file for writing:
>>> tif = TIFF.open('filename.tif', mode='w')
>>> # to write a image to tiff file
>>> tif.write_image(image)
Exemplo de uso (módulo Python puro)
>>> from libtiff import TIFFfile, TIFFimage
>>> # to open a tiff file for reading
>>> tif = TIFFfile('filename.tif')
>>> # to return memmaps of images and sample names (eg channel names, SamplesPerPixel>=1)
>>> samples, sample_names = tiff.get_samples()
>>> # to create a tiff structure from image data
>>> tiff = TIFFimage(data, description='')
>>> # to write tiff structure to file
>>> tiff.write_file('filename.tif', compression='none') # or 'lzw'
>>> del tiff # flushes data to disk
Exemplos de uso de script
$ libtiff.info -i result_0.tif
IFDEntry(tag=ImageWidth, value=512, count=1, offset=None)
IFDEntry(tag=ImageLength, value=512, count=1, offset=None)
IFDEntry(tag=BitsPerSample, value=32, count=1, offset=None)
IFDEntry(tag=Compression, value=1, count=1, offset=None)
IFDEntry(tag=PhotometricInterpretation, value=1, count=1, offset=None)
IFDEntry(tag=StripOffsets, value=8, count=1, offset=None)
IFDEntry(tag=Orientation, value=6, count=1, offset=None)
IFDEntry(tag=StripByteCounts, value=1048576, count=1, offset=None)
IFDEntry(tag=PlanarConfiguration, value=1, count=1, offset=None)
IFDEntry(tag=SampleFormat, value=3, count=1, offset=None)
Use --ifd to see the rest of 31 IFD entries
data is contiguous: False
memory usage is ok: True
sample data shapes and names:

width : 512
length : 512
samples_per_pixel : 1
planar_config : 1
bits_per_sample : 32
strip_length : 1048576

[('memmap', (32, 512, 512), dtype('float32'))] ['sample0']